## Analisando dados do Siconfi
*Por Thais Gabriele*

**Objetivo da análise**
* Conhecer os dados disponibilizados pelo Siconfi
* Investigar as **despesas** realizadas destinadas a área da Educação nos últimos anos

**Sumário**

1. Importando pacotes requeridos
2. Carregando a base de dados
3. Entendendo os dados
4. Visualização temporal


**Importando pacotes requeridos** 

In [1]:
import pandas as pd
import plotly.express as px
import basedosdados as bd 

**Carregando a base de dados**

In [2]:
# verificar base de dados da siconfi
bd.list_datasets(filter_by='siconfi', with_description=True)


dataset_id: 
	br_me_siconfi 
description: 
	Contém dados contábeis e fiscais enviados pelos entes da Federação sobre a sua 
orçamentária e financeira. 
-----------------------------------------------------------------------------------------------



In [3]:
# checar informações das tabelas da base de dados
bd.get_table_size(
    dataset_id='br_me_siconfi ',
    table_id='municipio_despesas_funcao',
    billing_project_id='educacao-342019'
)


project_id: 
	basedosdados 
dataset_id: 
	br_me_siconfi  
table_id: 
	municipio_despesas_funcao 
num_rows: 
	15126326 
size_mb: 
	0.0 
-----------------------------------------------------------------------------------------------



In [2]:
df = bd.read_sql(
        """ 
        SELECT * 
        FROM `basedosdados.br_me_siconfi.municipio_despesas_funcao`  
        WHERE estagio = 'Despesas Liquidadas' AND conta = 'Educação'
        """,
        billing_project_id='educacao-342019'
)

Downloading: 100%|██████████| 43314/43314 [00:05<00:00, 8614.57rows/s]


**Entendendo o contexto dos dados**
  
  * Os dados coletados fazem parte do conjunto de contas anuais

  * Classificadores de despesas:
      * Natureza da despesa:
          - Despesas empenhadas: quando há intenção de contratar
          - Despesas liquidadas: quando foi contratado e o serviço foi entregue
          - Despesas pagas: quanto foi pago ao fornecedor/prestador
          
      * Função/Subfunção:
          - educação, legislativo, saúde..
          
**Referências**:

    * https://www.youtube.com/watch?v=wZ2RAsQQWqA
    * https://meusiconfi.tesouro.gov.br/Interface_rsiconfi.rmd


**Explorando os dados**

In [3]:
# tamanho do df
df.shape

(43314, 10)

In [4]:
df.head()

,ano,sigla_uf,id_municipio,estagio,portaria,conta,estagio_bd,id_conta_bd,conta_bd,valor
0,2013,AC,1200252,Despesas Liquidadas,12,Educação,Despesas Liquidadas,3.12.000,Educação,8.433650e+06
1,2013,AC,1200401,Despesas Liquidadas,12,Educação,Despesas Liquidadas,3.12.000,Educação,1.064155e+08
2,2013,AC,1200104,Despesas Liquidadas,12,Educação,Despesas Liquidadas,3.12.000,Educação,1.047275e+07
3,2013,AC,1200609,Despesas Liquidadas,12,Educação,Despesas Liquidadas,3.12.000,Educação,2.366587e+07
4,2013,AC,1200393,Despesas Liquidadas,12,Educação,Despesas Liquidadas,3.12.000,Educação,8.706395e+06


In [5]:
df = df.rename(columns={'sigla_uf' : 'sigla'})

In [6]:
cols = ['ano','sigla','estagio','conta','valor'] # colunas necessárias

In [7]:
novo_df = df.filter(items=cols)

In [8]:
novo_df.head()

,ano,sigla,estagio,conta,valor
0,2013,AC,Despesas Liquidadas,Educação,8.433650e+06
1,2013,AC,Despesas Liquidadas,Educação,1.064155e+08
2,2013,AC,Despesas Liquidadas,Educação,1.047275e+07
3,2013,AC,Despesas Liquidadas,Educação,2.366587e+07
4,2013,AC,Despesas Liquidadas,Educação,8.706395e+06


In [9]:
novo_df.sigla.unique()

array(['AC', 'AP', 'RR', 'RO', 'SE', 'AM', 'ES', 'AL', 'MS', 'TO', 'PA',
       'GO', 'PI', 'PE', 'MT', 'MA', 'RN', 'RJ', 'PB', 'CE', 'SC', 'BA',
       'PR', 'RS', 'SP', 'MG'], dtype=object)

In [11]:
df_ac = novo_df.loc[novo_df.sigla == "AC"].groupby(['ano']).sum()

In [12]:
df_ac

,valor
ano,
2013,3.575569e+08
2014,3.850986e+08
2015,4.438613e+08
2016,4.769034e+08
2017,4.697921e+08
2018,4.886955e+08
2019,5.216669e+08
2020,4.937687e+08


In [13]:
novo_df.groupby(["sigla","ano"]).sum()

valor
sigla ano               
AC    2013  3.575569e+08
      2014  3.850986e+08
      2015  4.438613e+08
      2016  4.769034e+08
      2017  4.697921e+08
...                  ...
TO    2016  1.219172e+09
      2017  1.274890e+09
      2018  1.439034e+09
      2019  1.557020e+09
      2020  1.477466e+09

[208 rows x 1 columns]

In [15]:
df_agrup = novo_df.groupby(["sigla","ano"]).sum().reset_index()

In [16]:
df_agrup.head()

,sigla,ano,valor
0,AC,2013,3.575569e+08
1,AC,2014,3.850986e+08
2,AC,2015,4.438613e+08
3,AC,2016,4.769034e+08
4,AC,2017,4.697921e+08


**Visualização temporal dos dados**

1. Visualização dos Estados em geral

  - Qual o estado teve mais despesas com a educação nos ultimos anos?
  
  - Há uma discrepância entre os estados?
  
2. Visualização temporal mais detalhada

 - Como se deram essas despesas nos últimos anos?
 
 - Em algum estado diminuiu?

In [18]:
# somar o total das despesas de 2013 a 2020
df_total = novo_df.groupby("sigla").sum().sort_values("valor",ascending=False)

In [21]:
# gráfico com o total de despesas liquidadas de 2013 a 2020

fig = px.bar(df_total,  
             y='valor', 
             x=df_total.index,
             text_auto='.2s',
             title=" 2013-2020 - Total de despesas liquidadas na educação por Estado")

fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)

fig.show()

fig.write_html("geral.html")


*É possível notar uma discrepância muito grande entre São Paulo e os demais estados*

In [32]:
## 5 estados que mais tiveram despesas na educação
top5mais = (novo_df.groupby("sigla")
                    .sum()
                    .sort_values("valor",ascending=False)
                    .head(5))

display(top5mais)

,ano,valor
sigla,,
SP,10300339,2.927613e+11
MG,13494667,9.369397e+10
RJ,1441798,8.634136e+10
BA,6497208,8.515015e+10
RS,7912863,6.064745e+10


In [33]:
top5mais_sigla = top5mais.index.tolist()
print(top5mais_sigla)

['SP', 'MG', 'RJ', 'BA', 'RS']


In [35]:
## 5 estados que menos tiveram despesas na educação
top5menos = (novo_df.groupby("sigla")
                    .sum()
                    .sort_values("valor",ascending=True)
                    .head(5))

display(top5menos)

,ano,valor
sigla,,
AP,199639,2.901664e+09
RR,215790,3.042790e+09
AC,334735,3.637343e+09
RO,832816,8.212255e+09
TO,2183892,9.784317e+09


In [36]:
top5menos_sigla = top5menos.index.tolist()
print(top5menos_sigla)

['AP', 'RR', 'AC', 'RO', 'TO']


In [38]:
df_mais = novo_df[novo_df.sigla.isin(top5mais_sigla)]
df_mais = df_mais.groupby(["sigla","ano"]).sum().reset_index()

In [39]:
df_menos = novo_df[novo_df.sigla.isin(top5menos_sigla)]
df_menos = df_menos.groupby(["sigla","ano"]).sum().reset_index()

In [51]:
fig = px.line(df_mais, 
              x = df_mais.ano, 
              y = df_mais.valor,
              color = 'sigla', 
              symbol= "sigla", 
              title="Cinco estados com mais despesas liquidadas na educação"
        )

fig.update_xaxes(rangeslider_visible=True)

fig.show()

fig.write_html("topmais.html")


*Observa-se uma queda de 2019 para 2020*

In [50]:
fig = px.line(df_menos, 
              x = df_menos.ano, 
              y = df_menos.valor,
              color = "sigla", 
              symbol= "sigla", 
              title="Cinco estados com menos despesas liquidadas na educação"
             )

fig.update_xaxes(rangeslider_visible=True)

fig.show()

fig.write_html("topmenos.html")


*Também observa-se uma queda de 2019 para 2020. Porém diferentemente do gŕafico anterior há também uma queda no estado do Amapá e Roraima entre 2015 e 2016*

## Conclusões

*Analisando algumas matérias recentes é possível observar que os dados estão de acordo com as observações feitas nessa análise. Estados como São Paulo e Bahia se mantem no topo do ranking em investimentos não só na área da educação mas também em outras áreas. Sendo assim, o próximo passo dessa análise é correlacionar os valores observados com os índices educacionais e assim checar os reais efeitos das despesas. Além disso, também podemos verificar os outros tipos de despesas*

* https://bahia.ba/bahia/governo-do-estado-investiu-r-18-bilhoes-de-2015-a-2021/